# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
import pandas as pd

###10 Countries with lowest infant mortality rates

In [7]:
#Extract country and infant mortality data where available 
data = [] #using list of dicts good way to build dataframe? other ways? 

for element in document.iterfind('country'):
    c= element.find('name').text
    if element.find('infant_mortality') is not None:
        im = float(element.find('infant_mortality').text) #good idea to convert to numeric here ?
                                                          #or do converstion after putting in dataframe? 
        data.append({'country': c, 'infant_mortality' : im})

df = pd.DataFrame(data) 
df = df.set_index('country')
df.sort('infant_mortality').head(10)

,infant_mortality
country,
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13


###10 Cities with largest population

In [8]:
#Extract all population data, including year of data
data = []

for element in document.iterfind('country'):
    co= element.find('name').text
    for city in element.getiterator('city'):
        ci=city.find('name').text #note if city has multiple names this is taking first 
        for pop in city.getiterator('population'):
            p=float(pop.text) #right time to convert?
            y=pop.attrib['year']
            data.append({'country': co, 'city': ci, 'population' : p , 'year' : y})

df = pd.DataFrame(data)

In [9]:
#pick out latest year of data for each city, use that to find 10 largest cities 
#this is an issue I run into - is this the best approach? 
#how to create a function to make this more replicable?
bestpopbycity=df.groupby(['city','country'])['year'].max()
bestpopbycity = bestpopbycity.reset_index().rename(columns={'year':'best_year'})
df = df.merge(bestpopbycity).set_index('city')
pd.set_option('float_format','{:20,.0f}'.format) #found this but don't really understand the format notation
df[df.year==df.best_year].sort('population',ascending=False).iloc[0:10,0:3]

,country,population,year
city,,,
Shanghai,China,"22,315,474",2010
Istanbul,Turkey,"13,710,512",2012
Mumbai,India,"12,442,373",2011
Moskva,Russia,"11,979,529",2013
Beijing,China,"11,716,620",2010
São Paulo,Brazil,"11,152,344",2010
Tianjin,China,"11,090,314",2010
Guangzhou,China,"11,071,424",2010
Delhi,India,"11,034,555",2011


### 10 Ethnic groups with largest populations

In [10]:
#First get best population data for each country
data = []

for element in document.iterfind('country'):
    co= element.find('name').text
    for pop in element.iterfind('population'): #what's difference betwen iterfind and findall ? 
        p=float(pop.text) #right time to convert?
        y=pop.attrib['year']
        data.append({'country': co, 'population' : p , 'year' : y})

df = pd.DataFrame(data)

bestpopbycountry=df.groupby(['country'])['year'].max()
bestpopbycountry = bestpopbycountry.reset_index().rename(columns={'year':'best_year'})
df = df.merge(bestpopbycountry)
bestpopbycountry=df[df.year==df.best_year]
bestpopbycountry.head()

,country,population,year,best_year
8,Albania,"2,800,138",2011,2011
23,Greece,"10,816,286",2011,2011
34,Macedonia,"2,059,794",2011,2011
42,Serbia,"7,120,666",2011,2011
52,Montenegro,"620,029",2011,2011


In [11]:
#get ethnicity data
data = []

for element in document.iterfind('country'):
    co= element.find('name').text
    for egroup in element.iterfind('ethnicgroup'): #what's difference betwen iterfind and findall ? 
        e=egroup.text #right time to convert?
        p=float(egroup.attrib['percentage'])
        data.append({'country': co, 'ethnicgroup': e, 'percentage' : p})

ethnicgroups = pd.DataFrame(data)
ethnicgroups.head()

,country,ethnicgroup,percentage
0,Albania,Albanian,95
1,Albania,Greek,3
2,Greece,Greek,93
3,Macedonia,Macedonian,64
4,Macedonia,Albanian,25


In [12]:
#combine tables, calculate totals by ethnic group
merged = ethnicgroups.merge(bestpopbycountry).rename(columns={'population':'country_population'})
merged['population']=merged.country_population*merged.percentage/100
pd.set_option('float_format','{:20,.0f}'.format) #found this but don't really understand the format notation
pd.DataFrame(merged.groupby('ethnicgroup')['population'].sum().order(ascending=False)).head(10)

,population
ethnicgroup,
Han Chinese,"1,245,058,800"
Indo-Aryan,"871,815,583"
European,"494,872,220"
African,"318,325,120"
Dravidian,"302,713,744"
Mestizo,"157,734,355"
Bengali,"146,776,917"
Russian,"131,856,996"
Japanese,"126,534,212"
